In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle
import csv

from gpu import set_gpu

import tensorflow as tf
from adapt.ml.dl_subclass import MLP, MLPprop, CNN, get_train, get_test
import adapt.utils.data_utils as prd
from adapt.ml.lda import train_lda, eval_lda

set_gpu()

In [ ]:
path = 'C:/Users/yteh/Documents/work/necal/home data/TR58/DATA/MAT/'
train_file = '20170913_061042'
adapt_file = '20170913_061042'
test_file = '20170913_061042'

In [ ]:
train_data, train_params = prd.load_caps_train(path + train_file + '/traindata.mat')
train_data = train_data[:,:8,:]

In [ ]:
ep = 30
n_dof = np.max(train_params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_prop_accuracy = tf.keras.metrics.MeanSquaredError(name='train_prop_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(train_data, train_params)

# Train neural networks
models = [mlp, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    prop_b = isinstance(model, MLPprop) or isinstance(model, CNN)
 
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)

mlp_w = mlp.get_weights()
cnn_w = cnn.get_weights()

In [ ]:
adapt_data, adapt_params = prd.load_caps_train(path + adapt_file + '/traindata.mat')
adapt_data = adapt_data[:,:8,:]

In [ ]:
ep = 5
n_dof = np.max(adapt_params[:,0])

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_prop_accuracy = tf.keras.metrics.MeanSquaredError(name='train_prop_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(adapt_data, adapt_params)

# Train neural networks
models = [mlp, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    prop_b = isinstance(model, MLPprop) or isinstance(model, CNN)
 
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)

mlp_w = mlp.get_weights()
cnn_w = cnn.get_weights()

In [ ]:
test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
test_data = test_data[:,:8,:].astype('float64')
y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale)

test_mod = get_test()
test_mod(x_test_cnn, y_test, model, test_loss, test_accuracy)
print (f'Accuracy: {test_accuracy.result()*100:.2f}')

acc = eval_lda(w, c, x_lda, y_lda)
print (f'LDA Accuracy: {acc*100:.2f}')